In [ ]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [ ]:
%use kandy
%use dataframe

In [ ]:
import kotlin.random.Random

// Prepare the measurement data 
fun getRandomMeasure(
    trueValue: Double,
    range: Double
): DoubleArray {

    val measureVal = Random.nextDouble(trueValue - range, trueValue + range)
    return  doubleArrayOf(measureVal)
}

In [5]:
import org.apache.commons.math3.filter.KalmanFilter
import org.apache.commons.math3.filter.DefaultProcessModel
import org.apache.commons.math3.filter.ProcessModel
import org.apache.commons.math3.filter.DefaultMeasurementModel
import org.apache.commons.math3.filter.MeasurementModel


val processModel: ProcessModel = DefaultProcessModel(
    arrayOf( //stateTransition: X
        doubleArrayOf(1.0)
    ),
    arrayOf( // control : U
        doubleArrayOf(1.0)
    ),
    arrayOf( // processNoise: Q
        doubleArrayOf(1.0)
    ),
    doubleArrayOf(50.0), // inital state estimate
    arrayOf( // initial covariance
        doubleArrayOf(100.0*100.0)
    ),
)

val measurementModel = DefaultMeasurementModel(
    arrayOf( //measMatrix: H
        doubleArrayOf(1.0)
    ),
    arrayOf( //measNoise : R
        doubleArrayOf(100.0*100)
    ),
)

val kalmanFilter = KalmanFilter(processModel, measurementModel)


In [6]:

val trueValue = 100.0
val range = 100.0


val estimate = mutableListOf<Double>()
val measurement = mutableListOf<Double>()
val covariances = mutableListOf<Double>()


repeat(100){ i ->
    
    kalmanFilter.predict()

    kalmanFilter.stateEstimation
    
    
    kalmanFilter.correct(
            getRandomMeasure(
                trueValue = trueValue ,
                range = range
            ).also {
                measurement.add(it[0])
            }
        )
    
    estimate.add(kalmanFilter.stateEstimation[0])
    covariances.add(kalmanFilter.errorCovariance[0][0])

}


val plotData = dataFrameOf(
    "data" to estimate +  measurement,
    "type" to  List(estimate.size) { "estimate" } + List(measurement.size) { "measurement" },
    "index" to List(estimate.size) { it } +  List(measurement.size) { it }
)


plotData.plot {
    line {
        x("index")
        y("data")
        color("type"){
            scale = categorical("measurement" to Color.BLUE, "estimate" to Color.RED )
        }
    }
    layout.size = 1000 to 450

}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wQDY1x"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"data":[54.36683657750095,40.56309269076654,30.418871727147305,33.16907670038336,37.490602350779064,47.59072118162534,56.35571532178731,70.39991693668222,76.33774179336727,83.56346421261796,93.03134363653716,88.59930874226086,84.6048827188865,84.10093517840912,86.87837883130051,90.21114138263988,91.5451272768433,94.17364688074596,91.31983325349039,89.68373098726721,94.11443290984249,93.42511613974591,96.76350543338698,100.3982378365976,97.10367316770505,94.20019189010756,93.87427241222885,92.54104614854533,94.47869042791837,94.41616609755988,96.25034102563151,93.59332498991665,91.5901345509754,91.9306554985372,89.42004356573715,88.39038092472641,90.53900822823816,92.88846879878733,92.44708907670918,92.19299194866453,94.76983133785066,96.35382400716358,97.26017613756754,95.04677782896556,93.42199857050987,95.35418118582061,93.3386550219121,95.09476671093613,96.46303121326103,95.62528715685316,94.6901769729886,95.58266699631483,96.08612164543896,95.01784673541269,96.02162989103223,95.25843328342984,95.22867917109988,94.54498342640113,93.49259298752143,93.60433639710082,93.98053184569531,92.55683135519979,93.91341699471153,95.68333105758681,95.11179557047369,94.83053897920948,95.69676661495134,96.29059797094968,95.73075406501829,94.18155361198616,95.83502421786883,95.73391509629829,96.86747381006307,98.27019765110335,98.36474325454031,98.24058086179106,98.50363389413293,99.05165760257262,100.50812258124078,101.1238028871651,100.03913017719586,101.19091478207766,100.48675301588204,100.63473426323405,99.61907040071998,99.11047980572759,100.22565942101447,100.12720497264847,99.35655020400615,100.06101608945465,99.83042915359863,99.70389876904302,100.95042653818315,100.15540217233779,99.55759176391251,100.24971572102191,99.62739635038615,100.55058797109605,100.74692396393783,100.1296471798356,58.733236515008144,12.9625049952236,4.0364826310312907E-4,44.16165273831456,59.074491173413946,108.09967906613012,117.58824107709329,182.46785656830158,129.60954786575124,155.54372222030563,196.70146815647124,35.70118023257034,33.00211726617797,77.09639323617783,128.19891183375975,143.0421361848167,113.9876545368344,140.9401357959949,37.79170054930028,57.4234811516811,185.71880881098164,78.51654944322372,172.1341436929263,185.8925616733483,16.513655542243022,20.460933475935693,85.29367715820855,56.206800995250575,149.06847505551846,92.59743247176657,151.27220411340977,11.485250971110016,27.88685453635127,103.06353578754224,5.1123712047423275,52.906808000565285,166.46301348228582,177.949511067238,76.0866459708476,82.55706036362895,194.67732319770107,159.10176322203662,133.92166718282232,3.6812289393909348,25.018125843265192,178.27465321391415,5.212939537524575,173.28441255510774,158.47003403854254,57.00156014012598,50.848840437698925,138.11464832166902,120.46330769336457,42.483511307889074,146.13678893611782,56.588600105040186,93.69923873711944,58.904276517509935,37.87512394302017,99.58934550006768,114.39448442994585,14.309561671781035,169.40614656866023,195.3825304734634,62.532462593228445,78.61084522903927,146.220772918148,131.31274620794872,62.35341173645337,0.836396972865816,196.49971143330313,89.51567623139492,167.27515612648534,186.24378844388286,104.3506951025003,90.3063703777615,115.46618045778025,134.70463032803042,196.0869774986556,141.87132611168227,27.653950167041042,178.68168317639402,52.73343654267857,110.74858768736242,29.672393368446937,63.82285048035179,178.1669637681494,93.19677611745148,44.7277893216047,150.3409238068272,83.26205035028427,90.55244320874516,191.6881637012221,41.91839783989777,55.496227709601634,151.57114945936272,

In [ ]:
val convarianceDf = dataFrameOf(
    "covariances" to covariances,
    "index" to List(covariances.size) { it } 
)


convarianceDf.plot {
    line {
        x("index")
        y("covariances")
    }
}